- http://localhost:8887/edit/git/pytorch-tutorial/tutorials/03-advanced/image_captioning/model.py
- http://localhost:8887/edit/git/pytorch-tutorial/tutorials/03-advanced/image_captioning/train.py

# setup

In [1]:
from collections import OrderedDict

# run 1step of training

In [2]:
import argparse
import torch
import torch.nn as nn
import numpy as np
import os
import pickle
from data_loader import get_loader 
from build_vocab import Vocabulary
from model import EncoderCNN, DecoderRNN
from torch.nn.utils.rnn import pack_padded_sequence
from torchvision import transforms


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

parser = argparse.ArgumentParser()
parser.add_argument('--model_path', type=str, default='models/' , help='path for saving trained models')
parser.add_argument('--crop_size', type=int, default=224 , help='size for randomly cropping images')
parser.add_argument('--vocab_path', type=str, default='data/vocab.pkl', help='path for vocabulary wrapper')
parser.add_argument('--image_dir', type=str, default='data/resized2014', help='directory for resized images')
parser.add_argument('--caption_path', type=str, default='data/annotations/captions_train2014.json', help='path for train annotation json file')
parser.add_argument('--log_step', type=int , default=10, help='step size for prining log info')
parser.add_argument('--save_step', type=int , default=1000, help='step size for saving trained models')

# Model parameters
parser.add_argument('--embed_size', type=int , default=256, help='dimension of word embedding vectors')
parser.add_argument('--hidden_size', type=int , default=512, help='dimension of lstm hidden states')
parser.add_argument('--num_layers', type=int , default=1, help='number of layers in lstm')

parser.add_argument('--num_epochs', type=int, default=5)
parser.add_argument('--batch_size', type=int, default=16)
parser.add_argument('--num_workers', type=int, default=2)
parser.add_argument('--learning_rate', type=float, default=0.001)
args = parser.parse_args("")

In [3]:
# Create model directory
if not os.path.exists(args.model_path):
    os.makedirs(args.model_path)

# Image preprocessing, normalization for the pretrained resnet
transform = transforms.Compose([ 
    transforms.RandomCrop(args.crop_size),
    transforms.RandomHorizontalFlip(), 
    transforms.ToTensor(), 
    transforms.Normalize((0.485, 0.456, 0.406), 
                         (0.229, 0.224, 0.225))])

# Load vocabulary wrapper
with open(args.vocab_path, 'rb') as f:
    vocab = pickle.load(f)

# Build data loader
data_loader = get_loader(args.image_dir, args.caption_path, vocab, 
                         transform, args.batch_size,
                         shuffle=True, num_workers=args.num_workers) 

loading annotations into memory...
Done (t=0.51s)
creating index...
index created!


# encoder

In [4]:
images, captions, lengths = next(iter(data_loader))
# Set mini-batch dataset
images = images.to(device)
captions = captions.to(device)
targets = pack_padded_sequence(captions, lengths, batch_first=True)[0]

In [5]:
encoder = EncoderCNN(args.embed_size).to(device)

nmd_children = OrderedDict(encoder.named_children())

nmd_children.keys()

odict_keys(['resnet', 'linear', 'bn'])

In [13]:
images.size()

torch.Size([16, 3, 224, 224])

In [6]:
def print_ioshape(self, input, output):
    print('')
    print('Inside ' + self.__class__.__name__ + ' forward')
    print('')
    print('input type: ', type(input))
    print('input len:', len(input))
    print('input[0] type: ', type(input[0]))
    print('input[0] size: ', input[0].size())
    print('')
    print('output type: ', type(output))
    print('output size: ', output.size())
    print('')
    print('-'*10)


for child in nmd_children.values():
    child.register_forward_hook(print_ioshape)

features = encoder(images)


Inside Sequential forward

input type:  <class 'tuple'>
input len: 1
input[0] type:  <class 'torch.Tensor'>
input[0] size:  torch.Size([16, 3, 224, 224])

output type:  <class 'torch.Tensor'>
output size:  torch.Size([16, 2048, 1, 1])

----------

Inside Linear forward

input type:  <class 'tuple'>
input len: 1
input[0] type:  <class 'torch.Tensor'>
input[0] size:  torch.Size([16, 2048])

output type:  <class 'torch.Tensor'>
output size:  torch.Size([16, 256])

----------

Inside BatchNorm1d forward

input type:  <class 'tuple'>
input len: 1
input[0] type:  <class 'torch.Tensor'>
input[0] size:  torch.Size([16, 256])

output type:  <class 'torch.Tensor'>
output size:  torch.Size([16, 256])

----------


# decoder

In [15]:
decoder = DecoderRNN(args.embed_size, args.hidden_size, len(vocab), args.num_layers).to(device)

nmd_children = OrderedDict(decoder.named_children())

nmd_children

OrderedDict([('embed', Embedding(9957, 256)),
             ('lstm', LSTM(256, 512, batch_first=True)),
             ('linear',
              Linear(in_features=512, out_features=9957, bias=True))])

In [16]:
images, captions, lengths = next(iter(data_loader))
# Set mini-batch dataset
images = images.to(device)
captions = captions.to(device)
targets = pack_padded_sequence(captions, lengths, batch_first=True)[0]

encoder = EncoderCNN(args.embed_size).to(device)
features = encoder(images)

## embed

In [17]:
def print_ioshape(self, input, output):
    print('Inside ' + self.__class__.__name__ + ' forward')
    print('')
    print('input type: ', type(input))
    print('input len:', len(input))
    print('input[0] type: ', type(input[0]))
    print('input[0] size: ', input[0].size())
    print('')
    print('output type: ', type(output))
    print('output size: ', output.size())


decoder = DecoderRNN(args.embed_size, args.hidden_size, len(vocab), args.num_layers).to(device)
nmd_children = OrderedDict(decoder.named_children())
nmd_children['embed'].register_forward_hook(print_ioshape)
outputs = decoder(features, captions, lengths)

Inside Embedding forward

input type:  <class 'tuple'>
input len: 1
input[0] type:  <class 'torch.Tensor'>
input[0] size:  torch.Size([16, 20])

output type:  <class 'torch.Tensor'>
output size:  torch.Size([16, 20, 256])


## lstm

In [10]:
def print_ioshape(self, input, output):
    print('Inside ' + self.__class__.__name__ + ' forward')
    print('')
    print('input type: ', type(input))
    print('input len:', len(input))
    print('input[0] type: ', type(input[0]))
    print('input[0] len: ', len(input[0]))
    print('input[0][0] len: ', len(input[0][0]))
    print('input[0][1] len: ', len(input[0][1]))
    print('')
    print('output type: ', type(output))
    print('out len:', len(output))
    print('output[0] type: ', type(output[0]))
    print('output[0] len: ', len(output[0]))
    print('output[0][0] len: ', len(output[0][0]))
    print('output[0][1] len: ', len(output[0][1]))
    print('output[1] type: ', type(output[1]))
    print('output[1] len: ', len(output[1]))
    print('output[1][0] type: ', type(output[1][0]))
    print('output[1][0] size: ', output[1][0].size())
    print('output[1][1] type: ', type(output[1][1]))
    print('output[1][1] size: ', output[1][1].size())


decoder = DecoderRNN(args.embed_size, args.hidden_size, len(vocab), args.num_layers).to(device)
nmd_children = OrderedDict(decoder.named_children())
nmd_children['lstm'].register_forward_hook(print_ioshape)
outputs = decoder(features, captions, lengths)

Inside LSTM forward

input type:  <class 'tuple'>
input len: 1
input[0] type:  <class 'torch.nn.utils.rnn.PackedSequence'>
input[0] len:  2
input[0][0] len:  233
input[0][1] len:  19

output type:  <class 'tuple'>
out len: 2
output[0] type:  <class 'torch.nn.utils.rnn.PackedSequence'>
output[0] len:  2
output[0][0] len:  233
output[0][1] len:  19
output[1] type:  <class 'tuple'>
output[1] len:  2
output[1][0] type:  <class 'torch.Tensor'>
output[1][0] size:  torch.Size([1, 16, 512])
output[1][1] type:  <class 'torch.Tensor'>
output[1][1] size:  torch.Size([1, 16, 512])


## linear

In [11]:
def print_ioshape(self, input, output):
    print('Inside ' + self.__class__.__name__ + ' forward')
    print('')
    print('input type: ', type(input))
    print('input len:', len(input))
    print('input[0] type: ', type(input[0]))
    print('input[0] size: ', input[0].size())
    print('')
    print('output type: ', type(output))
    print('output size: ', output.size())

decoder = DecoderRNN(args.embed_size, args.hidden_size, len(vocab), args.num_layers).to(device)
nmd_children = OrderedDict(decoder.named_children())
nmd_children['linear'].register_forward_hook(print_ioshape)
outputs = decoder(features, captions, lengths)

Inside Linear forward

input type:  <class 'tuple'>
input len: 1
input[0] type:  <class 'torch.Tensor'>
input[0] size:  torch.Size([233, 512])

output type:  <class 'torch.Tensor'>
output size:  torch.Size([233, 9957])
